In [1]:
import numpy as np
import math
import cmath
import gudhi as gd
import pandas  as pd
import timeit

In [2]:
#copied from https://thispointer.com/python-how-to-append-a-new-row-to-an-existing-csv-file/

from csv import writer
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)
        

In [3]:
#DF of critical point cardinality. Fixed mass at 1. Vary number of masses from one to twenty.
#Pt mass model equal masses, positions based on Weyl Polynomials. 
start_time = timeit.default_timer()

csv_file_name = 'Weyl01.csv' 
with open("Weyl01.csv", "w") as my_empty_csv:
    pass
row_content = ['m','g','l','Saddles','Minima','Total Images']
append_list_as_row(csv_file_name,row_content)

df_gm_weyl=pd.DataFrame()
df_means=pd.DataFrame()
lrange=36
mrange=16
grange=21
for g in range(1,grange):
    for m in range(mrange):
        for l in range(lrange):
            #Random Model const mass uniform random locations in circle size sqrt g
            #Weyl polynomial version of random lensing. Uniform masses

            #location of source
            yy=complex(0,0)

            #mass of points
            mass=.5+m*.1

            #Weyl polynomial coefficients
            mu, sigma = 0,1 
            n=np.random.randn(g+1, 2).view(np.complex128) 
            c=np.ones(g+1)
            for i in range(g+1):
                c[i]=1/np.sqrt(np.math.factorial(i))
            c=c.reshape(g+1,1)
            N=np.multiply(c,n)   


            x=np.linspace(-2*np.sqrt(g),2*np.sqrt(g),1000)
            y=np.linspace(-2*np.sqrt(g),2*np.sqrt(g),1000)

            #two lists in R, that is a grid in R^2, to one list in C
            zz=np.zeros(len(x)*len(y),dtype=np.complex_)
            for i in range(len(y)):
                for j in range(len(x)):
                    zz[j+i*len(x)] = complex(x[j],y[i])        


            z=np.zeros((len(y),len(x)))          
            for i in range(len(y)):
                for j in range(len(x)):
                    #Quadratic part
                    Q=abs(zz[j+i*len(x)]-yy)**2/2
                    #Gravitational potential part
                    K=0
                    for k in range(g+1):
                        K=K+N[k]*zz[j+i*len(x)]**k             
                    G=m*np.log(abs(K))
                    z[i,j]=Q-G

            bincount=1000
            bins = np.linspace(z.min(),.5*z.max(),bincount)
            inds = np.digitize(z,bins)

            file = open("starpers.txt","w") 
            file.write(str(len(z.shape))+"\n")
            file.write(str(z.shape[0])+"\n") 
            file.write(str(z.shape[1])+"\n")

            for i in range(z.shape[0]):
                for j in range(z.shape[1]):
                    if  inds[i][j]==bincount:
                        file.write("inf"+"\n")
                    else:
                        file.write(str(inds[i][j])+"\n")

            file.close()

            cc = gd.CubicalComplex(perseus_file="starpers.txt")
            pers_cc = cc.persistence()   
            saddles = len(cc.persistence_intervals_in_dimension(1)) + len(cc.persistence_intervals_in_dimension(0)-1)
            minima = len(cc.persistence_intervals_in_dimension(0))
            df=pd.DataFrame()
            df['m']=[mass]
            df['g']=[g]
            df['i']=[l]
            df['Saddles']=[saddles]
            df['Minima']=[minima]
            df['Total Images']=[minima+saddles]
            df_gm_weyl=df_gm_weyl.append(df)
            row_content=[mass,g,l,saddles,minima,minima+saddles]
            append_list_as_row(csv_file_name,row_content)   
            elapsed = timeit.default_timer() - start_time
#            print(df)
#            print(elapsed)
#            print(m)
#            print(l)
        df=pd.DataFrame()    
        df1=pd.DataFrame()
        df2=pd.DataFrame()
        df3=pd.DataFrame()
        df1=df_gm_weyl[(df_gm_weyl['m']==mass)&(df_gm_weyl['g']==g)]['Total Images']
        df2=df_gm_weyl[(df_gm_weyl['m']==mass)&(df_gm_weyl['g']==g)]['Saddles']
        df3=df_gm_weyl[(df_gm_weyl['m']==mass)&(df_gm_weyl['g']==g)]['Minima']
        ImgMean=np.mean(df1.values)
        SadMean=np.mean(df2.values)
        MinMean=np.mean(df3.values)
        df['m']=[mass]
        df['g']=[g]
        df['Avg Critical Points']=[ImgMean]
        df['Avg Index 1 Critical Points']=[SadMean]
        df['Avg Index 0 Critical Points']=[MinMean]
        df_means=df_means.append(df)  
print(elapsed)
df_means.to_pickle('mWg1_20_m_05_01_2')
df_gm_weyl.to_pickle('Wg1_20m_05_01_2')
df_gm_weyl.to_csv('Weyl1.csv',index=False)

473651.498885504


In [4]:
df = pd.read_csv('Weyl0.csv')
df

,m,g,l,Saddles,Minima,Total Images
0,0.5,1,0,1,1,2
1,0.5,1,1,1,1,2
2,0.5,1,2,1,1,2
3,0.5,1,3,1,1,2
4,0.5,1,4,1,1,2
...,...,...,...,...,...,...
3821,1.5,7,5,10,4,14
3822,1.5,7,6,11,4,15
3823,1.5,7,7,11,4,15
3824,1.5,7,8,11,4,15
